# <span style="color:darkred">Additive Schwarz  using multiple strategies</span>.

<img align="left" src="Schwarz.jpg" width=120 height=120 />

### [Simon Tavener](https://www.math.colostate.edu/~tavener/) and Andre Leautaud

#### Full text for this notebook can be found at the following [link](https://www.math.colostate.edu/~tavener/andre/DD_multiphysics.pdf)

This notebook illustrates three ways of solving a system using Additive Schwarz. It first creates partitions and necessary matrices, then steps through the forms of iteration. It demonstrates C Iteration (each subdomain independently), D Iteration (subdomains calculated in parallel) and  U Iteration (computes all iterations as one matrix solve). In each, we keep track of total error, iteration error, and discretization error (true and estimates). If the estimates are good, and the system was solved, the effectivty ratio will be 1. It can read all parameters and the linear system from a csv. It can also create a csv. This csv is compatible with Multiplicative Schwarz and the MatLab Live files.

#### Set path and parameters

In [380]:
# load necessary packages
import numpy as np
import math
%run DD_Utilities.ipynb
%run AS_Utilities.ipynb
iprint= 4 # amount of output desired, 0, 2, 4, or 6

In [381]:
iparam= -2

In [382]:
if iparam == -2:
    n= 12
    p= 4 
    b= [1,2,5,9]
    e= [3,7,10,12] #List of ends (inclusive) or integer representing overlap
    K= 3
    alpha= 0.5
    mpert= 0.001
    if K > 500:
        raise ValueError("K is too large")

In [383]:
if iparam == 0:
    n= 10  # size of system
    p= 4   # number of subomains
    b= [1,2,4,7]  # indices of starts of subdomains (1 indexed), should be in ascending order.
    e= [3,4,8,10] # indices of ends of subdomains (1 indexed), need not be in ascending order. 
                  # Setting to an integer will make each subdomain overlap the next by that number.

    K= 3   # number of iterations
    alpha= 0.5 # alpha is only used in multiplicative schwarz. Including it here makes the file i/o compatible.
    mpert= 0.02
    if K > 500:
        raise ValueError("K is too large")

In [384]:
# This example shows how to create domains that each overlap by 1 column
#small system
if iparam == 1:
    n= 8
    p= 3 
    b= [1,3,5]
    e= 1 # causes each subdomain to overlap the previous by 1
    K= 3
    alpha= 0.5
    mpert= 0.02
    if K > 500:
        raise ValueError("K is too large")

In [385]:
# some subdomains embedded in others
if iparam == 2:
    n= 10
    p= 5
    b= [1,1,5,5,7]
    e= [2,4,6,8,10] 
    K= 6
    alpha= 0.5
    mpert= 0.02
    if K > 500:
        raise ValueError("K is too large")

In [386]:
# overlap does not connect all subdomains
if iparam == 3:
    n= 9
    p= 4 
    b= [1,3,6,8]
    e= [2,5,7,9]
    K= 3
    alpha= 0.5
    mpert= 0.02
    if K > 500:
        raise ValueError("K is too large")

In [387]:
if iparam == 4:
    n= 12
    p= 4 
    b= [1,2,5,9]
    e= [3,7,10,12]
    K= 3
    alpha= 0.5
    mpert= 0.02
    if K > 500:
        raise ValueError("K is too large")

In [388]:
print(p,b,e)
nn,nb,ne,index= fix_domains(n,p,b,e,iprint)

4 [1, 2, 5, 9] [3, 7, 10, 12]


### Establish and solve forward system, $Mx=y$

### Establish subdomains, partitions, number of iterations and random numerical error.  Read and/or save as appropriate

In [389]:
# Read and write capability. Usually only use one of sread and swrite, but it is acceptable to set both to True.
sread= False   #Read system of equations from iparam[iparam].csv. The solver will now ignore all of the parameters set above.
swrite= True    #Write system to csv file
file_name= 'table3_AS_effect_of_K'#i'param' + str(iparam) 

M,x,y,xstore= Linear_system(n,sread,file_name)
wpert= Make_wpert(n,p,K,sread,file_name) #Read or create wpert (depending on sread)

if iprint >= 4:
    print(M)
    print('y')
    print(y)
    print('x')
    print(x)

if swrite:
    save_to_csv(n,p,K,alpha,nb,nn,ne,M,y,mpert,wpert,iparam)

[[1.24170220e+01 7.20324493e-01 1.14374817e-04 3.02332573e-01
  1.46755891e-01 9.23385948e-02 1.86260211e-01 3.45560727e-01
  3.96767474e-01 5.38816734e-01 4.19194514e-01 6.85219500e-01]
 [2.04452250e-01 1.28781174e+01 2.73875932e-02 6.70467510e-01
  4.17304802e-01 5.58689828e-01 1.40386939e-01 1.98101489e-01
  8.00744569e-01 9.68261576e-01 3.13424178e-01 6.92322616e-01]
 [8.76389152e-01 8.94606664e-01 1.20850442e+01 3.90547832e-02
  1.69830420e-01 8.78142503e-01 9.83468338e-02 4.21107625e-01
  9.57889530e-01 5.33165285e-01 6.91877114e-01 3.15515631e-01]
 [6.86500928e-01 8.34625672e-01 1.82882773e-02 1.27501443e+01
  9.88861089e-01 7.48165654e-01 2.80443992e-01 7.89279328e-01
  1.03226007e-01 4.47893526e-01 9.08595503e-01 2.93614148e-01]
 [2.87775339e-01 1.30028572e-01 1.93669579e-02 6.78835533e-01
  1.22116281e+01 2.65546659e-01 4.91573159e-01 5.33625451e-02
  5.74117605e-01 1.46728575e-01 5.89305537e-01 6.99758360e-01]
 [1.02334429e-01 4.14055988e-01 6.94400158e-01 4.14179270e-01
  4

### Iterative strategy
$u^{\{k\}}= u^{\{k-1\}} + Br = u^{\{k-1\}} + B(f-Au^{\{k-1\}})$, $\quad k=1, \dots, K,$ where $B$ is an approximation to $A^{-1}$

<ins>Additive<ins>
$A_i= R_i A R_i^\top \in \mathbb{R}^{m_i \times m_i} \\
B_i= R_i^\top A_i^{-1} R_i  = R_i^\top ( R_i A R_i^\top)^{-1} R_i  \in \mathbb{R}^{n \times n} \\
C_i= B_i A   \in \mathbb{R}^{n \times n} \\
f_i= B_i f   \in \mathbb{R}^{n}
$

In [390]:
#Construct domains, R,A,B,C, and f
R= Rmatrices(n,p,nn,index,iprint)
A= Amatrices(M,R,n,p,nn,iprint)
B= Bmatrices(A,R,n,p,nn,iprint)
C= Cmatrices(M,B,n,p,iprint) 
f= fvector(B,y,n,p,iprint)

Print f vector 
i= 0
[0.00310865 0.00825975 0.01783988 0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]
i= 1
[ 0.          0.00427171  0.01773143  0.05238996  0.04285054 -0.00208064
  0.00108153  0.          0.          0.          0.          0.        ]
i= 2
[ 0.          0.          0.          0.          0.04381268 -0.00524394
 -0.00460419  0.07103158  0.04115988  0.00632209  0.          0.        ]
i= 3
[0.         0.         0.         0.         0.         0.
 0.         0.         0.03913504 0.01206122 0.01482181 0.05936004]


####  $K$ iterations computing subdomain contributions independently [C iteration]
$
u^{\{k\}}= u^{\{k-1\}} + \alpha \sum_{i=1}^p B_i(f-Au^{\{k-1\}}) = u^{\{k-1\}} + \alpha \sum_{i=1}^p ( f_i-C_iu^{\{k-1\}}) 
$

In [391]:
#Perform C iteration
vstore= Citeration_additive(C,f,alpha,n,p,K,iprint)

Outer iteration 0
[[ 0.00155432  0.00626573  0.01778566  0.02619498  0.04333161 -0.00366229
  -0.00176133  0.03551579  0.04014746  0.00919165  0.0074109   0.02968002]]
Outer iteration 1
[[-0.0002635  -0.00022079  0.01239888  0.03773649  0.03982141 -0.00731294
  -0.00544885  0.05161378  0.03711612  0.00589707  0.00860193  0.04133049]]
Outer iteration 2
[[-0.00167516 -0.0007625   0.01216898  0.04300103  0.03898027 -0.00750185
  -0.00575557  0.05947514  0.0365869   0.00622488  0.00884402  0.04708328]]


In [392]:
if iprint >= 4:
    print('vstore')
    print(vstore)

vstore
[[ 0.00155432]
 [ 0.00626573]
 [ 0.01778566]
 [ 0.02619498]
 [ 0.04333161]
 [-0.00366229]
 [-0.00176133]
 [ 0.03551579]
 [ 0.04014746]
 [ 0.00919165]
 [ 0.0074109 ]
 [ 0.02968002]
 [-0.0002635 ]
 [-0.00022079]
 [ 0.01239888]
 [ 0.03773649]
 [ 0.03982141]
 [-0.00731294]
 [-0.00544885]
 [ 0.05161378]
 [ 0.03711612]
 [ 0.00589707]
 [ 0.00860193]
 [ 0.04133049]
 [-0.00167516]
 [-0.0007625 ]
 [ 0.01216898]
 [ 0.04300103]
 [ 0.03898027]
 [-0.00750185]
 [-0.00575557]
 [ 0.05947514]
 [ 0.0365869 ]
 [ 0.00622488]
 [ 0.00884402]
 [ 0.04708328]]


####  $K$ iterations combining $p$  (parallel) subdomain solves [D iteration]

<ins>Additive Schwarz<ins>
    
$u^{\{k\}}= D u^{\{k-1\}} + g, \qquad 
D = I-\alpha \sum_{i=1}^p C_i, \quad g= \alpha \sum_{i=1}^p f_i$


In [393]:
# Perform D iteration
#Construct D and g and solve
D= Dmatrix_additive(C,alpha,n,p,iprint)

dmatrix_additive:  matrix
[[ 5.00e-01  0.00e+00  0.00e+00 -1.07e-02 -5.00e-03 -2.50e-03 -7.20e-03
  -1.35e-02 -1.42e-02 -1.95e-02 -1.62e-02 -2.61e-02]
 [-6.10e-03  0.00e+00  0.00e+00 -2.59e-02 -1.61e-02 -2.16e-02 -5.30e-03
  -1.26e-02 -5.95e-02 -7.29e-02 -2.00e-02 -5.18e-02]
 [-3.55e-02 -0.00e+00  0.00e+00  1.10e-03 -5.50e-03 -3.46e-02 -3.20e-03
  -3.14e-02 -7.09e-02 -3.56e-02 -5.17e-02 -1.97e-02]
 [-2.56e-02 -0.00e+00 -0.00e+00  5.00e-01  0.00e+00 -0.00e+00 -0.00e+00
  -2.91e-02  2.10e-03 -1.33e-02 -3.10e-02 -7.30e-03]
 [-2.00e-02 -2.70e-03  3.00e-04 -2.63e-02 -0.00e+00  0.00e+00 -0.00e+00
   1.00e-03 -2.14e-02 -3.60e-03 -4.28e-02 -5.41e-02]
 [-9.00e-04 -1.09e-02 -2.63e-02 -1.33e-02 -0.00e+00  0.00e+00 -0.00e+00
  -1.71e-02 -3.32e-02 -1.97e-02 -6.53e-02 -5.20e-03]
 [-3.70e-03 -2.65e-02 -1.40e-02 -8.00e-04  0.00e+00 -0.00e+00  0.00e+00
  -2.69e-02 -2.93e-02 -2.07e-02 -5.08e-02 -1.85e-02]
 [-7.70e-03 -3.15e-02 -1.50e-02 -3.45e-02  0.00e+00 -0.00e+00 -0.00e+00
   5.00e-01  0.00e+00 -0.00

In [394]:
g= gvector_additive(f,alpha,n,p,iprint)

Print g vector
[[ 0.00155432]
 [ 0.00626573]
 [ 0.01778566]
 [ 0.02619498]
 [ 0.04333161]
 [-0.00366229]
 [-0.00176133]
 [ 0.03551579]
 [ 0.04014746]
 [ 0.00919165]
 [ 0.0074109 ]
 [ 0.02968002]]


In [395]:
# print('Perform D iteration \n')
wstore= Diteration_additive(D,g,n,p,K,iprint)

Diteration_additive: w at iteration  0
[[ 0.26245446]
 [-0.10621657]
 [-0.10916646]
 [ 0.38861181]
 [-0.06299547]
 [-0.10342547]
 [-0.09042532]
 [ 0.06811668]
 [-0.09965691]
 [-0.09760452]
 [ 0.44768618]
 [ 0.14797892]]
Diteration_additive: w at iteration  1
[[ 0.12115002]
 [-0.00611142]
 [-0.00455306]
 [ 0.19792186]
 [ 0.00348138]
 [-0.03099587]
 [-0.0210558 ]
 [ 0.0528555 ]
 [ 0.01049061]
 [-0.00351662]
 [ 0.23172528]
 [ 0.10292634]]
Diteration_additive: w at iteration  2
[[ 0.05300041]
 [-0.0098753 ]
 [-0.00146041]
 [ 0.11264707]
 [ 0.02006873]
 [-0.02308063]
 [-0.01746341]
 [ 0.0510227 ]
 [ 0.02169413]
 [-0.00299844]
 [ 0.12101849]
 [ 0.07650595]]
norm(D)=  1.153997e+00
Asymptotic solution: z= inv(I-D)@g
K=     3, norm(w-z)=  1.494198e-01


In [396]:
if iprint >= 2:
    print('wstore')
    print(wstore)

wstore
[[ 0.26245446]
 [-0.10621657]
 [-0.10916646]
 [ 0.38861181]
 [-0.06299547]
 [-0.10342547]
 [-0.09042532]
 [ 0.06811668]
 [-0.09965691]
 [-0.09760452]
 [ 0.44768618]
 [ 0.14797892]
 [ 0.12115002]
 [-0.00611142]
 [-0.00455306]
 [ 0.19792186]
 [ 0.00348138]
 [-0.03099587]
 [-0.0210558 ]
 [ 0.0528555 ]
 [ 0.01049061]
 [-0.00351662]
 [ 0.23172528]
 [ 0.10292634]
 [ 0.05300041]
 [-0.0098753 ]
 [-0.00146041]
 [ 0.11264707]
 [ 0.02006873]
 [-0.02308063]
 [-0.01746341]
 [ 0.0510227 ]
 [ 0.02169413]
 [-0.00299844]
 [ 0.12101849]
 [ 0.07650595]]


#### Perform $K$ iterations as single matrix solve [U iteration]

$K=6$ iterations of multiplicative Schwarz can be written as the $nK$-dimensional system of equations

<ins>Additive Schwarz<ins>

$Uz=
\pmatrix{
 I & 0 & 0 & 0 & 0 & 0   \cr
 -D &I & 0 & 0 & 0 & 0   \cr
 0 & -D & I & 0 & 0 & 0   \cr
 0 & 0 & -D & I & 0 & 0   \cr
 0 & 0 & 0 & -D & I & 0   \cr
 0 & 0 & 0 & 0 & -D & I
}$
$\pmatrix{
v^{\{1\}}  \cr
v^{\{2\}}  \cr
v^{\{3\}}  \cr
v^{\{4\}}  \cr
v^{\{5\}}  \cr
v^{\{6\}}
}$
$\pmatrix{
g  \cr
g  \cr
g  \cr
g  \cr
g  \cr
g
}
=h$

In [397]:
# Perform U iteration
# Construct U and h
U= Umatrix_additive(D,n,K,iprint)
h= hvector_additive(g,n,K,iprint)

U matrix
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
h vector
[[ 0.00155432]
 [ 0.00626573]
 [ 0.01778566]
 [ 0.02619498]
 [ 0.04333161]
 [-0.00366229]
 [-0.00176133]
 [ 0.03551579]
 [ 0.04014746]
 [ 0.00919165]
 [ 0.0074109 ]
 [ 0.02968002]
 [ 0.00155432]
 [ 0.00626573]
 [ 0.01778566]
 [ 0.02619498]
 [ 0.04333161]
 [-0.00366229]
 [-0.00176133]
 [ 0.03551579]
 [ 0.04014746]
 [ 0.00919165]
 [ 0.0074109 ]
 [ 0.02968002]
 [ 0.00155432]
 [ 0.00626573]
 [ 0.01778566]
 [ 0.02619498]
 [ 0.04333161]
 [-0.00366229]
 [-0.00176133]
 [ 0.03551579]
 [ 0.04014746]
 [ 0.00919165]
 [ 0.0074109 ]
 [ 0.02968002]]


In [398]:
# Solve using Uz= h
zstore= Uhsolve_additive(U,h,n,K,iprint)

Uhsolve_additive: z at iteration 0
[ 0.00155432  0.00626573  0.01778566  0.02619498  0.04333161 -0.00366229
 -0.00176133  0.03551579  0.04014746  0.00919165  0.0074109   0.02968002]
Uhsolve_additive: z at iteration 1
[-0.0002635  -0.00022079  0.01239888  0.03773649  0.03982141 -0.00731294
 -0.00544885  0.05161378  0.03711612  0.00589707  0.00860193  0.04133049]
Uhsolve_additive: z at iteration 2
[-0.00167516 -0.0007625   0.01216898  0.04300103  0.03898027 -0.00750185
 -0.00575557  0.05947514  0.0365869   0.00622488  0.00884402  0.04708328]


In [399]:
if iprint >= 4:
    print('zstore')
    print(zstore)

zstore
[[ 0.00155432]
 [ 0.00626573]
 [ 0.01778566]
 [ 0.02619498]
 [ 0.04333161]
 [-0.00366229]
 [-0.00176133]
 [ 0.03551579]
 [ 0.04014746]
 [ 0.00919165]
 [ 0.0074109 ]
 [ 0.02968002]
 [-0.0002635 ]
 [-0.00022079]
 [ 0.01239888]
 [ 0.03773649]
 [ 0.03982141]
 [-0.00731294]
 [-0.00544885]
 [ 0.05161378]
 [ 0.03711612]
 [ 0.00589707]
 [ 0.00860193]
 [ 0.04133049]
 [-0.00167516]
 [-0.0007625 ]
 [ 0.01216898]
 [ 0.04300103]
 [ 0.03898027]
 [-0.00750185]
 [-0.00575557]
 [ 0.05947514]
 [ 0.0365869 ]
 [ 0.00622488]
 [ 0.00884402]
 [ 0.04708328]]


#### Compare Solutions from all three methods

In [400]:
solution_compare(xstore,vstore,wstore,zstore,iprint)

norm(exact - iterative) solutions    =  1.033501e-02
Difference between vstore and wstore =  7.855266e-01
Difference between wstore and zstore =  7.855266e-01
Difference between vstore and zstore =  2.023615e-17


### Add discretization error and compute error estimate

#### C iteration with error

In [401]:
psi= np.ones((n,1))

# Solve system with "numerical error" for every solve
vglobal_store,vaglobal_store,rglobal_store= Citeration_additive_approx(C,f,alpha,n,p,K,mpert,wpert,iprint)

if iprint >= 4:
    print('vglobal_store')
    print(vglobal_store)
    print('vaglobal_store')
    print(vaglobal_store)
    print('rglobal_store')
    print(rglobal_store)    

Citeration_additive_approx: Outer iteration 0
Difference = 2.8918e-03
dmatrix_additive:  matrix
[[ 5.00e-01  0.00e+00  0.00e+00 -1.07e-02 -5.00e-03 -2.50e-03 -7.20e-03
  -1.35e-02 -1.42e-02 -1.95e-02 -1.62e-02 -2.61e-02]
 [-6.10e-03  0.00e+00  0.00e+00 -2.59e-02 -1.61e-02 -2.16e-02 -5.30e-03
  -1.26e-02 -5.95e-02 -7.29e-02 -2.00e-02 -5.18e-02]
 [-3.55e-02 -0.00e+00  0.00e+00  1.10e-03 -5.50e-03 -3.46e-02 -3.20e-03
  -3.14e-02 -7.09e-02 -3.56e-02 -5.17e-02 -1.97e-02]
 [-2.56e-02 -0.00e+00 -0.00e+00  5.00e-01  0.00e+00 -0.00e+00 -0.00e+00
  -2.91e-02  2.10e-03 -1.33e-02 -3.10e-02 -7.30e-03]
 [-2.00e-02 -2.70e-03  3.00e-04 -2.63e-02 -0.00e+00  0.00e+00 -0.00e+00
   1.00e-03 -2.14e-02 -3.60e-03 -4.28e-02 -5.41e-02]
 [-9.00e-04 -1.09e-02 -2.63e-02 -1.33e-02 -0.00e+00  0.00e+00 -0.00e+00
  -1.71e-02 -3.32e-02 -1.97e-02 -6.53e-02 -5.20e-03]
 [-3.70e-03 -2.65e-02 -1.40e-02 -8.00e-04  0.00e+00 -0.00e+00  0.00e+00
  -2.69e-02 -2.93e-02 -2.07e-02 -5.08e-02 -1.85e-02]
 [-7.70e-03 -3.15e-02 -1.50e-

#### Calculate the discretization error and effectivity ratio

In [402]:
vglobal_store

array([[ 0.00155432],
       [ 0.00626573],
       [ 0.01778566],
       [ 0.02619498],
       [ 0.04333161],
       [-0.00366229],
       [-0.00176133],
       [ 0.03551579],
       [ 0.04014746],
       [ 0.00919165],
       [ 0.0074109 ],
       [ 0.02968002],
       [-0.0002635 ],
       [-0.00022079],
       [ 0.01239888],
       [ 0.03773649],
       [ 0.03982141],
       [-0.00731294],
       [-0.00544885],
       [ 0.05161378],
       [ 0.03711612],
       [ 0.00589707],
       [ 0.00860193],
       [ 0.04133049],
       [-0.00167516],
       [-0.0007625 ],
       [ 0.01216898],
       [ 0.04300103],
       [ 0.03898027],
       [-0.00750185],
       [-0.00575557],
       [ 0.05947514],
       [ 0.0365869 ],
       [ 0.00622488],
       [ 0.00884402],
       [ 0.04708328]])

In [403]:
# Adjoint solve
phiglobal_store= Citeration_additive_adjoint(C,psi,alpha,n,p,K,iprint)
if iprint >= 4:
    print('phiglobal_store')
    print(phiglobal_store.T)

# Error estimate= (r,\phi)
qoi_discretization_error_estimate= (rglobal_store.T @ phiglobal_store)[0,0]

if iprint >= 4:
    print(f'norm(r_store) = {np.linalg.norm(rglobal_store):13.6e}, norm(phi_store)= {np.linalg.norm(phiglobal_store):13.6e}')

# Compute error and error in QoI directly
vsoln= vglobal_store[n * (K-1): n * K, 0]
vasoln= vaglobal_store[n * K: n * (K+1), 0]
verror= vsoln.T-vasoln
qoi_discretization_error= (psi.T @ verror.T)[0]

# Calculate effectivity ratio
effectivity_ratio_disc= qoi_discretization_error_estimate/ qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error:13.6e}')
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc:13.3f}')

12 4 3
Citeration_additive_adjoint: k= 2, i= 3
[0.15332825 0.11463677 0.07106557 0.08760632 0.2108783  0.15430232
 0.16860121 0.19473913 1.         1.         1.         1.        ]
Citeration_additive_adjoint: k= 2, i= 2
[0.15081677 0.24738492 0.11390261 0.19213226 1.         1.
 1.         1.         1.         1.         0.2238931  0.1574815 ]
Citeration_additive_adjoint: k= 2, i= 1
[0.16210901 1.         1.         1.         1.         1.
 1.         0.18551736 0.29004651 0.22133345 0.28434186 0.1667762 ]
Citeration_additive_adjoint: k= 2, i= 0
[1.         1.         1.         0.07092999 0.05311856 0.11720092
 0.03134729 0.07364059 0.16260073 0.14933216 0.10924136 0.12342156]
Citeration_additive_adjoint: k= 1, i= 3
[-0.01261636 -0.02342669  0.01640859 -0.00366911  0.01331022 -0.00171316
  0.01192662  0.01358772 -0.22632362 -0.1853328   0.19126184  0.27616037]
Citeration_additive_adjoint: k= 1, i= 2
[-0.02242832 -0.01247378 -0.00214157 -0.00147113 -0.13199843 -0.13575162
 -0.09997

##### Calculate the total error and effectivity ratio

In [404]:
# Adjoint solve
phi= np.linalg.solve(M.T,psi)

# Residual
r= y.T - M @ vasoln

# Error estimate= (r,\phi)
qoi_total_error_estimate= (r @ phi)[0,0]

# Compute error in QoI directly
total_error= xstore.reshape(-1) - vasoln.T
qoi_total_error= psi.reshape(-1)@total_error

# Calculate effectivity ratio
effectivity_ratio_total= qoi_total_error_estimate / qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total:13.6e}')

QoI_total_error                   =  9.207482e-03
QoI_total_error_estimate          =  9.207482e-03
Effectivity ratio                 =  1.000000e+00


#### Calculate iteration error as the difference between the total error and the discretization error

In [405]:
qoi_iteration_error= qoi_total_error-qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error:13.6e}')

QoI_iteration_error               =  1.000206e-02


#### D iteration with error

In [406]:
# Diteration with error as a check on the accumulation of error
print('Perform D iteration with error')

Perform D iteration with error


In [407]:
w_store,wa_store,wr_store= Diteration_additive_approx(D,g,alpha,n,p,K,wpert,mpert,iprint);
print(f'norm(w_store-vglobal_store)= {np.linalg.norm(w_store-vglobal_store):13.6e}')

norm(w_store-vglobal_store)=  1.462990e-17


In [408]:
print(f'norm(wa_store-vaglobal_store)= {np.linalg.norm(wa_store-vaglobal_store):13.6e}')

norm(wa_store-vaglobal_store)=  2.488120e-17


In [409]:
print(f'norm(wr_store-rglobal_store)= {np.linalg.norm(wr_store-rglobal_store):13.6e}')

norm(wr_store-rglobal_store)=  2.177736e-17


### U iteration with error

In [410]:
print('Solving U system with error')

Solving U system with error


In [411]:
zexact,zapprox,zresid= Uhsolve_additive_approx(U,h,n,p,K,alpha,wpert,mpert,iprint)

In [412]:
if iprint >= 4:
    print('zexact')
    print(zexact.T)
    print('zapprox')
    print(zapprox.T)
    print('zresid')
    print(zresid.T)

zexact
[[ 0.00155432  0.00626573  0.01778566  0.02619498  0.04333161 -0.00366229
  -0.00176133  0.03551579  0.04014746  0.00919165  0.0074109   0.02968002
  -0.0002635  -0.00022079  0.01239888  0.03773649  0.03982141 -0.00731294
  -0.00544885  0.05161378  0.03711612  0.00589707  0.00860193  0.04133049
  -0.00167516 -0.0007625   0.01216898  0.04300103  0.03898027 -0.00750185
  -0.00575557  0.05947514  0.0365869   0.00622488  0.00884402  0.04708328]]
zapprox
[[ 0.00278929  0.00458446  0.0175928   0.02686138  0.04445698 -0.00343467
  -0.00219216  0.0354128   0.04122836  0.00961162  0.0065999   0.02956869
   0.00056959  0.00016032  0.01325211  0.03788093  0.04095363 -0.00620389
  -0.00470343  0.05049756  0.03654842  0.00573543  0.00955247  0.04208567
  -0.00080997 -0.00221728  0.01243041  0.04294671  0.03707936 -0.00686625
  -0.00743973  0.05996924  0.03759422  0.00516975  0.01057794  0.04886091]]
zresid
[[-0.00123497  0.00168127  0.00019285 -0.0006664  -0.00112537 -0.00022762
   0.0004308

In [413]:
if iprint >= 2:
    print(f'Norm of difference between zexact   and vglobal_store  {np.linalg.norm(zexact-vglobal_store):13.6e}')
    print(f'Norm of difference between zapprox  and vaglobal_store {np.linalg.norm(zapprox-vaglobal_store[n:]):13.6e}')
    print(f'Norm of difference between zresid   and rglobal_store  {np.linalg.norm(zresid-rglobal_store):13.6e}')

Norm of difference between zexact   and vglobal_store   2.023615e-17
Norm of difference between zapprox  and vaglobal_store  1.338203e-03
Norm of difference between zresid   and rglobal_store   1.249673e-03


#### Calculate the discretization error and effectivity ratio

In [414]:
# Adjoint data (Adjoint system is of size nK)
psi_nK= np.zeros((n*K,1))
psi_nK[n * (K-1):n * K,0:1]= psi

# Solve adjoint equation
phi_nK= np.linalg.solve(U.T,psi_nK)
if iprint >= 2:
    print(f'Norm of difference between phi_nK and phiglobal_store {np.linalg.norm(phi_nK-phiglobal_store):13.6e}')

Norm of difference between phi_nK and phiglobal_store  4.107688e-16


In [415]:
# Compute error estimate
qoi_discretization_error_estimate= (phi_nK.T @ zresid)[0,0]

# Compute error and error in QoI directly
zsoln= zexact[-n:,0:1]
zasoln= zapprox[-n:,0:1]
zerror= zsoln - zasoln
qoi_discretization_error= (psi.T @ zerror)[0,0]

#### Calculate the effectivity ratio

In [416]:
effectivity_ratio_disc= qoi_discretization_error_estimate / qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error:13.6e}' )
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc:13.3f}')

QoI_discretization_error          = -6.258717e-04
QoI_discretization_error_estimate = -6.258717e-04
Effectivity ratio                 =         1.000


#### Calculate the total error and effectivity ratio

In [417]:
# Residual
r= y - M @ zasoln

# Error estimate= (r,\phi)
qoi_total_error_estimate= r.T @ phi

# Compute error in QoI directly
total_error= xstore - zasoln
qoi_total_error= psi.T @ total_error

#Effectivity ratio
effectivity_ratio_total= qoi_total_error_estimate / qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error[0,0]:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total[0,0]:13.3f}')

QoI_total_error                   =  9.376192e-03
QoI_total_error_estimate          =  9.376192e-03
Effectivity ratio                 =         1.000


#### Calculate the iteration error as the difference between the total error and the discretization error

In [418]:
qoi_iteration_error= qoi_total_error - qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error[0,0]:13.6e}')

QoI_iteration_error               =  1.000206e-02
